In [41]:
import re
from pathlib import Path

import openpyxl
import polars as pl
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer

# HELPERS

In [42]:
def ruta_parquet(nombre_archivo: str) -> Path:
    ruta = f"data/{nombre_archivo}.parquet"
    rutafinal = Path().parent / ruta
    print(f"Ruta final: {rutafinal}")
    return rutafinal

def cargar_parquet(nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet) 
    return pl.read_parquet(ruta).lazy()

def guardar_parquet(data: pl.LazyFrame | pl.DataFrame, nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet)
    if isinstance(data, pl.LazyFrame):
        data.sink_parquet(ruta)
        return cargar_parquet(nombre_parquet)
    else:
        data.write_parquet(ruta)
        return cargar_parquet(nombre_parquet)

In [43]:
IMPRIMIR = True

def imprimir(mensaje: str, parametros=None, collect_lazy_frame=False, **kwargs):
    if not IMPRIMIR:
        return
    # Normaliza la fuente de valores: dict explícito o kwargs
    values = {}
    if isinstance(parametros, dict):
        values.update(parametros)
    values.update(kwargs)

    # Soporte opcional para LazyFrame
    if collect_lazy_frame:
        for k, v in list(values.items()):
            if hasattr(v, "collect"):
                values[k] = v.collect()

    print(mensaje.format(**values))

In [44]:

# DEFINICIONES INICIALES
ruta = Path().parent / "data/opiniones.xlsx"
print(f"Ruta del archivo Excel: {ruta} [exists: {ruta.exists()}]")

workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames


def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index].lower().replace(" ", "_")
    return nombre_hoja


def calificaciones(calificacion:str) -> int:
    """Convierte la calificación de string a int."""
    calificacion_map = {
        "Pésimo": 1,
        "Malo": 2,
        "Regular": 3,
        "Muy bueno": 4,
        "Excelente": 5
    }
    return calificacion_map.get(calificacion, 0)


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col


def añadir_columna_lugar_turistico(iteracion: int) -> pl.LazyFrame:
    df_excel = pl.read_excel(ruta, sheet_id=iteracion+1).lazy()
    nombre_hoja = obtener_nombre_hoja(iteracion)
    
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico"),
            pl.col("Calificación").map_elements(calificaciones).alias("calificacion_numerica"),
        ])
    )

    df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.collect_schema().names()})
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel




Ruta del archivo Excel: data/opiniones.xlsx [exists: True]


# Limpiezsa y tratamiento del dataset

In [45]:
lista_df: list[pl.LazyFrame] = []
for i, hoja in enumerate(hojas):
    lista_df.append(añadir_columna_lugar_turistico(i))

df_excel = juntar_df(lista_df)

# GUARDAR PARQUET
df_reseñas = guardar_parquet(df_excel, "reseñas_turisticas")


imprimir(f"{df_reseñas.collect()['opinión'][:5]}")
MENSAJE = f" Columnas: {df_reseñas.collect().columns}"
imprimir(F"{"="*len(MENSAJE)}")
imprimir(MENSAJE)
imprimir(F"{"="*len(MENSAJE)}")


Ruta final: data/reseñas_turisticas.parquet
Ruta final: data/reseñas_turisticas.parquet
shape: (5,)
Series: 'opinión' [str]
[
	""Basílica muy bien conservada,…
	""The Basilica (Guanajuato does…
	""Edificio de la iglesia amaril…
	""A must see place in town and …
	""Not particularly impressive, …
]
 Columnas: ['género', 'edad', 'nacional_ó_internacional', 'calificación', 'escala', 'número_de_aportaciones', 'título_de_la_opinión', 'opinión', 'país', 'idioma', 'dispositivo', 'fecha', 'lugar_turistico', 'calificacion_numerica']


# CONTRUCCION DEL VOCABULARIO, DISTRIBUCIÓN FRECUENCIAS

In [46]:
class Texto:
    def __init__(self, lazy_frame: pl.LazyFrame, TOP_PALABRAS:int | None = None):
        self.SOS = "<s>"
        self.EOS = "</s>"
        self.UNK = "<unk>"
        self.tokenizer = TweetTokenizer()
        self.lazy_frame = lazy_frame
        self.TOP_PALABRAS = TOP_PALABRAS

        # Guardamos estados
        self.freq_dist = None
        self.final_vocab = None
        

    
    def _tokenizar(self, texto: str) -> list[str]:
        tokens = self.tokenizer.tokenize(texto.lower().strip())
        return tokens

    def _tokenizar_data(self) -> pl.LazyFrame:
        df_limpio = (
            self.lazy_frame
            .drop_nulls(subset=["opinión"])
            .with_columns(
                pl.col("opinión")
                .cast(pl.Utf8)
                .str.strip_chars()                                 # recorta espacios
                .str.replace_all(r'^[“”"\'«»\(\)\[\]\s]+', '')      # limpia INICIO
                .str.replace_all(r'[“”"\'«»\(\)\[\]\s]+$', '')      # limpia FINAL
                .alias("opinión_limpia")
            )
            .filter(pl.col("opinión_limpia").str.len_chars() > 0)


            .with_columns(
                pl.col("opinión_limpia")
                .map_elements(self._tokenizar, return_dtype=pl.List(pl.Utf8))
                .alias("opinión_tokenizada")
            )
            .filter(pl.col("opinión_tokenizada").list.len() > 0)
        )
        return df_limpio

    
    # ============== VOCABULARIO FINAL ==============
    # def vocabulario_final(self):
    #     if self.freq_dist is None:
    #         self.distribuir_frecuencias()   
        
    #     vocab = [self.SOS, self.EOS, self.UNK]
    #     vocab += [palabra for palabra, _ in self.freq_dist]
    #     self.final_vocab = vocab
    #     return self.final_vocab
    
    
    # ============================================================
    # ============= OPCIÓN A – POLARS PURO =======================
    # ============================================================
    def distribuir_frecuencias_A(self):
        lf_tokens = self._tokenizar_data()

        lf_counts = (
            lf_tokens
            .select(pl.col("opinión_tokenizada"))
            .explode("opinión_tokenizada")
            .group_by("opinión_tokenizada")
            .len()
            .rename({"len": "frecuencia"})
            .sort("frecuencia", descending=True)
        )

        df_top = (
            lf_counts.limit(self.TOP_PALABRAS).collect(engine="streaming")
            if self.TOP_PALABRAS else lf_counts.collect(engine="streaming")
        )

        self.freq_dist = df_top
        return df_top


    # ============================================================
    # ============= OPCIÓN B – LISTA DE TUPLAS ===================
    # ============================================================
    def distribuir_frecuencias_B(self):
        lf_tokens = self._tokenizar_data()

        lf_counts = (
            lf_tokens
            .select(pl.col("opinión_tokenizada"))
            .explode("opinión_tokenizada")
            .group_by("opinión_tokenizada")
            .len()
            .rename({"len": "frecuencia"})
            .sort("frecuencia", descending=True)
        )

        df_top = (
            lf_counts.limit(self.TOP_PALABRAS).collect(engine="streaming")
            if self.TOP_PALABRAS else lf_counts.collect(engine="streaming")
        )

        tokens = df_top["opinión_tokenizada"].to_list()
        counts = df_top["frecuencia"].to_list()
        self.freq_dist = list(zip(tokens, counts))
        return self.freq_dist


    # ============================================================
    # ============= OPCIÓN C – DICCIONARIO + ID ==================
    # ============================================================
    def distribuir_frecuencias_C(self):
        lf_tokens = self._tokenizar_data()

        lf_counts = (
            lf_tokens
            .select(pl.col("opinión_tokenizada"))
            .explode("opinión_tokenizada")
            .group_by("opinión_tokenizada")
            .len()
            .rename({"len": "frecuencia"})
            .sort("frecuencia", descending=True)
            .with_row_count("token_id")
        )

        df_top = (
            lf_counts.limit(self.TOP_PALABRAS).collect(engine="streaming")
            if self.TOP_PALABRAS else lf_counts.collect(engine="streaming")
        )

        freq_dict = dict(zip(df_top["opinión_tokenizada"].to_list(), df_top["frecuencia"].to_list()))
        id_dict = dict(zip(df_top["opinión_tokenizada"].to_list(), df_top["token_id"].to_list()))

        self.freq_dist = freq_dict
        self.id_map = id_dict
        return freq_dict

## Limpieza
Se esta utilizando una limpieza de codigo de manera aislada y preparada, primero insertando token de inicio y de cierre preparando para poder hacer uso del mismo data set
Lista de cosas:
- Token de inicio y fin de oración

In [47]:
# Asegúrate de tener df_reseñas (LazyFrame) cargado antes de esto.
texto = Texto(df_reseñas)

In [48]:
# ---- A: Polars puro
print("=== Opción A ===")
df_frec = texto.distribuir_frecuencias_A()
print(df_frec)

=== Opción A ===
shape: (23_315, 2)
┌────────────────────┬────────────┐
│ opinión_tokenizada ┆ frecuencia │
│ ---                ┆ ---        │
│ str                ┆ u32        │
╞════════════════════╪════════════╡
│ .                  ┆ 25953      │
│ ,                  ┆ 22328      │
│ de                 ┆ 17338      │
│ la                 ┆ 13161      │
│ the                ┆ 11691      │
│ …                  ┆ …          │
│ calling            ┆ 1          │
│ guanadjuato        ┆ 1          │
│ impatient          ┆ 1          │
│ memebers           ┆ 1          │
│ tolerated          ┆ 1          │
└────────────────────┴────────────┘


In [49]:
# ---- B: Lista de tuplas
print("\n=== Opción B ===")
freq_list = texto.distribuir_frecuencias_B()
print(freq_list)


=== Opción B ===
[('.', 25953), (',', 22328), ('de', 17338), ('la', 13161), ('the', 11691), ('a', 9976), ('y', 9594), ('que', 9502), ('es', 7678), ('el', 7551), ('en', 7183), ('and', 5269), ('un', 5200), ('of', 5064), ('to', 4716), ('no', 4234), ('una', 3898), ('is', 3622), ('para', 3481), ('por', 3464), ('guanajuato', 3456), ('se', 3434), ('los', 3409), ('muy', 3379), ('las', 3130), ('lugar', 2946), ('in', 2904), ('con', 2709), ('del', 2675), ('you', 2548), ('it', 2420), ('lo', 2402), ('ciudad', 2098), ('pero', 1924), ('this', 1893), ('al', 1843), ('hay', 1719), ('si', 1708), ('!', 1704), ('te', 1666), ('i', 1639), ('su', 1555), ('historia', 1549), ('for', 1528), ('este', 1436), ('are', 1424), ('was', 1407), ('como', 1366), ('museo', 1354), ('más', 1344), ('there', 1312), ('but', 1311), ('me', 1216), ('esta', 1161), ('that', 1134), ('with', 1115), ('vista', 1107), ('on', 1103), ('(', 1042), ('o', 1037), ('we', 1002), ('not', 996), (')', 996), ('ver', 953), ('...', 934), ('teatro', 91

In [50]:
# ---- C: Diccionario + ID
print("\n=== Opción C ===")
freq_dict = texto.distribuir_frecuencias_C()
print(list(freq_dict.items()))
print("ID de 'museo':", texto.id_map.get("museo"))


=== Opción C ===


/var/folders/j4/21ypxx3x0019wgj7rqnlm2340000gn/T/ipykernel_61172/3595419796.py:122: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("token_id")


[('.', 25953), (',', 22328), ('de', 17338), ('la', 13161), ('the', 11691), ('a', 9976), ('y', 9594), ('que', 9502), ('es', 7678), ('el', 7551), ('en', 7183), ('and', 5269), ('un', 5200), ('of', 5064), ('to', 4716), ('no', 4234), ('una', 3898), ('is', 3622), ('para', 3481), ('por', 3464), ('guanajuato', 3456), ('se', 3434), ('los', 3409), ('muy', 3379), ('las', 3130), ('lugar', 2946), ('in', 2904), ('con', 2709), ('del', 2675), ('you', 2548), ('it', 2420), ('lo', 2402), ('ciudad', 2098), ('pero', 1924), ('this', 1893), ('al', 1843), ('hay', 1719), ('si', 1708), ('!', 1704), ('te', 1666), ('i', 1639), ('su', 1555), ('historia', 1549), ('for', 1528), ('este', 1436), ('are', 1424), ('was', 1407), ('como', 1366), ('museo', 1354), ('más', 1344), ('there', 1312), ('but', 1311), ('me', 1216), ('esta', 1161), ('that', 1134), ('with', 1115), ('vista', 1107), ('on', 1103), ('(', 1042), ('o', 1037), ('we', 1002), ('not', 996), (')', 996), ('ver', 953), ('...', 934), ('from', 915), ('teatro', 915),